In [1]:

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, pearsonr, chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os

# Set visualization style
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.2)

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)
os.makedirs('visualizations', exist_ok=True)

# Try different possible locations for the dataset
possible_paths = [
    'DatasetForCoffeeSales2.csv',  # Same directory
    './DatasetForCoffeeSales2.csv',  # Explicit current directory
    '../DatasetForCoffeeSales2.csv',  # Parent directory
    'data/DatasetForCoffeeSales2.csv'  # Data subdirectory
]

file_path = None
for path in possible_paths:
    if os.path.exists(path):
        file_path = path
        print(f"Found dataset at: {path}")
        break

if file_path is None:
    raise FileNotFoundError("Could not find DatasetForCoffeeSales2.csv in any of the expected locations. "
                           "Please place the CSV file in the same directory as this notebook, "
                           "or in a 'data' subdirectory.")

# Load the dataset
coffee_df = pd.read_csv(file_path, on_bad_lines='skip')

# Display basic info about the dataset
print("Dataset shape:", coffee_df.shape)
print("\nFirst 5 rows:")
print(coffee_df.head())

# Check column data types and missing values
print("\nDataset information:")
coffee_df.info()

# Check summary statistics
print("\nSummary statistics:")
print(coffee_df.describe())

# Check for duplicate rows
print(f"\nNumber of duplicate rows: {coffee_df.duplicated().sum()}")

# Check unique values for categorical columns
print("\nUnique Cities:", coffee_df['City'].unique())
print("\nUnique Categories:", coffee_df['Category'].unique())
print("\nUnique Products:", coffee_df['Product'].unique())
print("\nUnique Unit Prices:", coffee_df['Unit Price'].unique())

# Check distribution of boolean column
print("\nDiscount Usage Distribution:")
print(coffee_df['Used_Discount'].value_counts())

# Convert date string to datetime
coffee_df['Date'] = pd.to_datetime(coffee_df['Date'])

# Check for missing values
print("\nMissing values in each column:")
print(coffee_df.isnull().sum())

# Verify Sales Amount calculation (Unit Price * Quantity)
coffee_df['Calculated_Sales'] = coffee_df['Unit Price'] * coffee_df['Quantity']
print("\nVerifying Sales Amount calculation:")
print("Max difference:", (coffee_df['Sales Amount'] - coffee_df['Calculated_Sales']).abs().max())

# Verify Final Sales calculation (Sales Amount - Discount_Amount)
coffee_df['Calculated_Final'] = coffee_df['Sales Amount'] - coffee_df['Discount_Amount']
print("Max difference in Final Sales:", (coffee_df['Final Sales'] - coffee_df['Calculated_Final']).abs().max())

# Drop verification columns
coffee_df = coffee_df.drop(columns=['Calculated_Sales', 'Calculated_Final'])

# Extract time-based features
coffee_df['Year'] = coffee_df['Date'].dt.year
coffee_df['Month'] = coffee_df['Date'].dt.month
coffee_df['Day'] = coffee_df['Date'].dt.day
coffee_df['Day_of_week'] = coffee_df['Date'].dt.dayofweek
coffee_df['Quarter'] = coffee_df['Date'].dt.quarter

# Create price categories based on unit prices
price_map = {
    30: 'Budget',
    35: 'Standard',
    40: 'Premium',
    45: 'Luxury'
}
coffee_df['Price_Category'] = coffee_df['Unit Price'].map(price_map)

# Create a more general bean category (Premium vs Standard)
coffee_df['Bean_Category'] = coffee_df['Unit Price'].apply(
    lambda x: 'Premium' if x >= 40 else 'Standard')

# Rename Product column for clarity
coffee_df['Coffee_Bean_Type'] = coffee_df['Product']

# Display the enriched dataset
print("\nEnriched dataset (first 5 rows):")
print(coffee_df.head())

# Print summary information about the prepared dataset
print("\nSummary of Prepared Dataset:")
print(f"Number of records: {coffee_df.shape[0]}")
print(f"Number of features: {coffee_df.shape[1]}")
print(f"Time period: {coffee_df['Date'].min().strftime('%Y-%m-%d')} to {coffee_df['Date'].max().strftime('%Y-%m-%d')}")
print(f"Number of unique customers: {coffee_df['Customer_ID'].nunique()}")
print(f"Number of unique cities: {coffee_df['City'].nunique()}")
print(f"Price categories: {', '.join(coffee_df['Price_Category'].unique())}")
print(f"Coffee bean types: {', '.join(coffee_df['Coffee_Bean_Type'].unique())}")

# Save the preprocessed dataset for further analysis
coffee_df.to_csv('data/coffee_sales_prepared.csv', index=False)
print("Prepared dataset saved to 'data/coffee_sales_prepared.csv'")


Found dataset at: DatasetForCoffeeSales2.csv
Dataset shape: (730, 11)

First 5 rows:
       Date  Customer_ID    City      Category     Product  Unit Price  \
0  1/1/2023           32  Riyadh  coffee beans   Colombian          40   
1  1/2/2023           49    Abha  coffee beans  Costa Rica          35   
2  1/3/2023           75   Tabuk  coffee beans  Costa Rica          35   
3  1/4/2023           80    Abha  coffee beans   Ethiopian          45   
4  1/5/2023           78    Hail  coffee beans   Colombian          40   

   Quantity  Sales Amount  Used_Discount  Discount_Amount  Final Sales  
0        14           560          False                0          560  
1        17           595          False                0          595  
2        19           665          False                0          665  
3         1            45          False                0           45  
4        46          1840           True              368         1472  

Dataset information:
<class 'pa